#**Setup**

In [1]:
!pip install gradio
!pip install langchain
!pip install langchain-community
!pip install langchain-openai

In [2]:
import gradio as gr
import torch
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI

#**Image Classification**

In [3]:
# car_model = torch.load('cars_model.pt')
# car_model.eval()

In [4]:
# def predict(image):
#   with torch.no_grad():
#     pred = car_model(image)
#     print(pred)
#   return pred

def predict(image):
  return 'car body type'

#**LLM**

In [5]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"

os.environ["OPENAI_API_BASE"] = "API_BASE"

os.environ["OPENAI_API_KEY"] = "API_KEY"




from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(

    openai_api_version="2024-02-01",

    azure_deployment="deployment-testing",

)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://orionopenai.openai.azure.com/ to https://orionopenai.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then 

In [6]:
#Define json schema
car_schema = {
  "type": "function",
  "function": {
      "name": "car_attributes",
      "description": "Record some identifying attributes of a car.",
      "parameters": {
          "type": "object",
          "properties": {
              "car": {
                  "description": "The attributes of the car",
                  "type": "object",
                  "properties": {
                      "color": {
                          "description": "The color of the car",
                          "type": "string"
                          },
                      "brand": {
                          "description": "The brand name of the car",
                          "type": "string"
                          },
                      "model": {
                          "description": "The model name of the car",
                          "type": "string"
                          },
                      "manufactured_year": {
                          "description": "The year the car was manufactured",
                          "type": "integer"
                          },
                      "motor_size_cc": {
                          "description": "The motor size of the car",
                          "type": "integer"
                          },
                      "tires": {
                          "description": "The attributes of the tires",
                          "type": "object",
                          "properties": {
                              "type": {
                                  "description": "The tires type either new or used",
                                  "type": "string"
                                  },
                              "manufactured_year": {
                                  "description": "The year the tires were manufactured",
                                  "type": "integer"
                                  }
                              },
                          "required": [
                              "type",
                              "manufactured_year"
                              ]
                          },
                      "windows": {
                          "description": "The type of car windows",
                          "type": "string"
                          },
                      "notices": {
                          "description": "Notices or remarks or aditional information",
                          "type": "array",
                          "items":
                           {
                               "type": "object",
                               "properties": {
                                   "type": {
                                       "description": "The type of notice or remark or aditional information",
                                        "type": "string"
                                        },
                                   "description": {
                                       "description": "The notice or remark or aditional information",
                                       "type": "string"
                                       }
                                   },
                               "required": [
                                   "type",
                                   "description"
                                   ]
                               }
                          },
                      "price": {
                          "description": "Price of the car",
                          "type": "object",
                          "properties": {
                              "amount": {
                                  "description": "The price of the car",
                                  "type": "integer"
                                  },
                              "currency": {
                                  "description": "The currency of price of the car",
                                  "type": "string"
                                  }
                              },
                          "required": [
                              "amount",
                              "currency"
                              ]
                          }
                      },
                  "required": [
                      "body_type",
                      "color",
                      "brand",
                      "model",
                      "manufactured_year",
                      "motor_size_cc",
                      "tires",
                      "windows",
                      "notices",
                      "price"
                      ]
                  }
              },
          "required": [
              "car"
              ]
          }
      }
  }


In [7]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm."
            "Extract the car attributes from the car description text."
            "Only extract relevant information from the car description text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{user_input}"),
    ]
)

In [8]:
def llm(text):
  structured_llm = create_structured_output_runnable(
      car_schema,
      chat,
      mode="openai-tools",
      enforce_function_usage=True,
      return_single=True
      )
  prompt = template.invoke({"user_input": text})
  llm_output = structured_llm.invoke(prompt)

  return llm_output

#**Gradio**

In [9]:
def car_description(text, image):
  #predict body type
  body_type = predict(image)
  #extract car attributes
  llm_json = llm(text)
  #Add body type to extraced attributes
  new_dictionary = {"body_type": body_type}
  new_dictionary.update(llm_json['car'])
  new_dict = {'car':new_dictionary}

  return new_dict

In [10]:
interface = gr.Interface(fn=car_description, inputs=[gr.Textbox(lines=5, placeholder="Type car description"), gr.components.Image()],
                     outputs=gr.JSON(), title="Online car-selling platform",
                     description="Display car attributes in JSON format")

# Launch the interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cf35dceae288b69344.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
!pip install pipreqsnb

In [13]:
!pipreqsnb .

pipreqs  .
INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in ./requirements.txt
